In [16]:
import numpy as np
import math

In [17]:
import numpy as np
import math

def prepare_state(amplitudes):
    """
    Prepares a quantum state vector from a list of complex amplitudes.

    Args:
        amplitudes (list[complex]): A list of complex or real amplitudes for the quantum state.

    Returns:
        np.ndarray: A normalized NumPy array representing the quantum state vector.
    """

    num_ampl = len(amplitudes)

    #Input Size Validation
    if num_ampl == 0 or (num_ampl & (num_ampl - 1)) != 0:
        raise ValueError(
            f"The number of amplitudes must be a power of 2, but got {num_ampl}."
        )

    state_vector = np.array(amplitudes, dtype=np.complex128)

    #Normalization
    norm_sq = np.sum(np.abs(state_vector)**2)

    if not np.isclose(norm_sq, 1.0):
        if np.isclose(norm_sq, 0.0):
             raise ValueError("Can't normalize a zero-vector")
        norm = np.sqrt(norm_sq)
        state_vector = state_vector / norm
        print(f"Info: Input amplitudes (norm^2={norm_sq:.4f}) were not normalized. Normalization has been applied.")

    return state_vector

In [18]:
def tensor_product(a, b):
    '''
    Calculates the Tensor Product  of two vectors, replicates the functionality of np.kron
    '''
    a_2d = _to_2d(a)
    b_2d = _to_2d(b)

    m = len(a_2d)
    n = len(a_2d[0]) if m > 0 else 0
    p = len(b_2d)
    q = len(b_2d[0]) if p > 0 else 0

    if m == 0 or n == 0 or p == 0 or q == 0:
        return [[]] * (m * p)

    result_rows = m * p
    result_cols = n * q

    result = [[0 for _ in range(result_cols)] for _ in range(result_rows)]

    for i in range(m):
        for j in range(n):
            scalar = a_2d[i][j]
            row_start = i * p
            col_start = j * q

            for k in range(p):
                for l in range(q):
                    val = scalar * b_2d[k][l]
                    target_row = row_start + k
                    target_col = col_start + l
                    result[target_row][target_col] = val

    return np.array(result)

def _to_2d(x):
    if isinstance(x, np.ndarray):
        if x.ndim == 0: return [[x.item()]]
        elif x.ndim == 1: return [x.tolist()]
        elif x.ndim == 2: return x.tolist()
        else: raise ValueError("Input NumPy array must be 0D, 1D, or 2D.")

    if isinstance(x, (int, float)): return [[x]]

    if isinstance(x, list):
        if not x: return []
        if all(isinstance(item, (int, float)) for item in x): return [x]
        if all(isinstance(item, list) for item in x): return x

    raise ValueError("Input must be a scalar, 1D/2D list, or 1D/2D NumPy array.")


In [19]:
def build_product_state(*amplitude_lists):
    """
    Constructs a composite quantum state by taking the tensor product of two or more single-qubit states

    Args:
        *amplitude_lists (list[complex]): Variable number of amplitude lists.

    Returns:
        np.ndarray: N-qubit state vector.
    """
    if not amplitude_lists:
        raise ValueError("At least one amplitude list must be provided.")

    #Normalize first state
    final_state_vector = prepare_state(amplitude_lists[0])

    #Iteratively compute tensor product
    for amps in amplitude_lists[1:]:
        qubit_vector = prepare_state(amps)
        final_state_vector = tensor_product(final_state_vector, qubit_vector)

    return final_state_vector.reshape(-1)

In [20]:
def get_gate_definitions():
    """
    Returns the matrix–vector representations of fundamental quantum gates.
    """
    I_1 = np.array([[1, 0], [0, 1]], dtype=np.complex128)
    H_1 = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=np.complex128)
    X_1 = np.array([[0, 1], [1, 0]], dtype=np.complex128)
    P0_1 = np.array([[1, 0], [0, 0]], dtype=np.complex128)
    P1_1 = np.array([[0, 0], [0, 1]], dtype=np.complex128)

    return {
        'I': I_1,
        'H': H_1,
        'X': X_1,
        'P0': P0_1,
        'P1': P1_1
    }


In [21]:
def build_bell_state():
    """
    Constructs the Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2 from scratch.
    """
    gates = get_gate_definitions()
    I, H = gates['I'], gates['H']

    # Initial state |00⟩
    psi_0 = build_product_state([1, 0], [1, 0])
    # Build the Hadamard gate on qubit 0: H ⊗ I
    H_0 = tensor_product(H, I)

    # Build the CNOT gate
    CNOT_01 = tensor_product(gates['P0'], I) + tensor_product(gates['P1'], gates['X'])

    # Apply gates
    psi_1 = H_0 @ psi_0.T
    psi_final = CNOT_01 @ psi_1

    return psi_final

In [22]:
def build_ghz_state():
    """
    Constructs the 3-Qubit GHZ state = (|000⟩ + |111⟩)/√2.
    """
    gates = get_gate_definitions()
    I, H, X, P0, P1 = gates['I'], gates['H'], gates['X'], gates['P0'], gates['P1']

    # Initial state |000⟩
    psi_0 = prepare_state([1, 0, 0, 0, 0, 0, 0, 0])

    # Build H ⊗ I ⊗ I
    H_0 = tensor_product(H, tensor_product(I, I))

    # Build CNOTs
    CNOT_01_base = tensor_product(P0, I) + tensor_product(P1, X)
    CNOT_01_3q = tensor_product(CNOT_01_base, I)
    CNOT_02_3q = tensor_product(P0, tensor_product(I, I)) + tensor_product(P1, tensor_product(I, X))

    psi_1 = H_0 @ psi_0
    psi_2 = CNOT_01_3q @ psi_1
    psi_final = CNOT_02_3q @ psi_2

    return psi_final

In [23]:
# Example usage for 2-Qubit state

unnormalized_amps_2q = [1, 0, 1, 0]
state_vector_2q = prepare_state(unnormalized_amps_2q)
print("Validated/Normalized State", end = '\n')
print(f"Input: {unnormalized_amps_2q}")
print("Resulting State Vector:\n", state_vector_2q, end = '\n\n')

print("Product State |0⟩ ⊗ |+⟩")
prod_state = build_product_state([1, 0], [1, 1])
print("Resulting State Vector:\n", prod_state,  end = '\n\n')

print("Constructed Bell State |Φ+⟩")
bell_state = build_bell_state()
print("Resulting State Vector:\n", bell_state,  end = '\n\n')

# Example usage for 3-Qubit state
print("Constructed GHZ State")
ghz_state = build_ghz_state()
print("Resulting State Vector:\n", ghz_state,  end = '\n\n')


Info: Input amplitudes (norm^2=2.0000) were not normalized. Normalization has been applied.
Validated/Normalized State
Input: [1, 0, 1, 0]
Resulting State Vector:
 [0.70710678+0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]

Product State |0⟩ ⊗ |+⟩
Info: Input amplitudes (norm^2=2.0000) were not normalized. Normalization has been applied.
Resulting State Vector:
 [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]

Constructed Bell State |Φ+⟩
Resulting State Vector:
 [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]

Constructed GHZ State
Resulting State Vector:
 [0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]



In [24]:
import unittest
import numpy as np

class TestStatePreparation(unittest.TestCase):
    """
    Unit tests for the prepare_state function and state construction functions.
    """

    def test_normalization_is_enforced(self):
        """
        Checks if an unnormalized input vector is correctly normalized.
        """
        unnormalized_amplitudes = [1, 1, 0, 0]
        state_vector = prepare_state(unnormalized_amplitudes)
        norm = np.sum(np.abs(state_vector)**2)
        self.assertAlmostEqual(norm, 1.0, places=7, msg="State vector should be normalized to 1.")

    def test_already_normalized_input(self):
        """
        Checks if a pre-normalized vector remains unchanged.
        """
        normalized_amplitudes = [1/np.sqrt(2), 0, -1/np.sqrt(2), 0]
        state_vector = prepare_state(normalized_amplitudes)
        norm = np.sum(np.abs(state_vector)**2)
        self.assertAlmostEqual(norm, 1.0, places=7, msg="Already normalized vector should have norm 1.")

        expected_vector = np.array([1/np.sqrt(2), 0, -1/np.sqrt(2), 0], dtype=np.complex128)
        np.testing.assert_array_almost_equal(state_vector, expected_vector, decimal=7)

    def test_output_dimension_two_qubits(self):
        """
        Ensures the output vector for a 2-qubit system has the correct dimension (4).
        """
        amplitudes = [0.5, 0.5, 0.5, 0.5]
        state_vector = prepare_state(amplitudes)
        self.assertEqual(len(state_vector), 4, msg="2-qubit state vector should have 4 elements.")

    def test_invalid_input_size(self):
        """
        Checks if a ValueError is raised for an input with a size that is not a power of 2.
        """
        amplitudes = [1, 2, 3]
        with self.assertRaises(ValueError):
            prepare_state(amplitudes)

    def test_zero_vector_input(self):
        """
        Checks if a ValueError is raised for a zero-vector input, which cannot be normalized.
        """
        amplitudes = [0, 0, 0, 0]
        with self.assertRaises(ValueError):
            prepare_state(amplitudes)

    def test_normalization_one_qubit(self):
        """
        Tests normalization for a single qubit.
        """
        state_vector = prepare_state([1, 1])
        expected = np.array([1/np.sqrt(2), 1/np.sqrt(2)], dtype=np.complex128)
        np.testing.assert_array_almost_equal(state_vector, expected, decimal=7)
        self.assertAlmostEqual(np.sum(np.abs(state_vector)**2), 1.0)


    def test_build_product_state_2q(self):
        """
        Tests constructing a 2-qubit product state |0⟩ ⊗ |+⟩.
        Inputs are unnormalized.
        """
        amps1 = [1, 0]
        amps2 = [1, 1]
        state_vector = build_product_state(amps1, amps2)

        expected = np.array([1/np.sqrt(2), 1/np.sqrt(2), 0, 0], dtype=np.complex128)
        np.testing.assert_array_almost_equal(state_vector, expected, decimal=7)
        self.assertEqual(len(state_vector), 4)

    def test_build_product_state_3q_stretch(self):
        """
        (Stretch Goal) Tests constructing a 3-qubit product state |1⟩ ⊗ |0⟩ ⊗ |1⟩.
        """
        amps1 = [0, 1]
        amps2 = [1, 0]
        amps3 = [0, 1]
        state_vector = build_product_state(amps1, amps2, amps3)

        expected = np.array([0, 0, 0, 0, 0, 1, 0, 0], dtype=np.complex128)
        np.testing.assert_array_almost_equal(state_vector, expected, decimal=7)
        self.assertEqual(len(state_vector), 8)

    def test_build_bell_state_construction(self):
        """
        Tests the construction of the Bell state |Φ+⟩ via gates.
        """
        state_vector = build_bell_state()
        expected = np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], dtype=np.complex128)
        np.testing.assert_array_almost_equal(state_vector, expected, decimal=7)
        self.assertAlmostEqual(np.sum(np.abs(state_vector)**2), 1.0)

    def test_build_ghz_state_construction_stretch(self):
        """
        (Stretch Goal) Tests the construction of the 3-Qubit GHZ state via gates.
        """
        state_vector = build_ghz_state()
        expected = np.array([1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 1/np.sqrt(2)], dtype=np.complex128)
        np.testing.assert_array_almost_equal(state_vector, expected, decimal=7)
        self.assertAlmostEqual(np.sum(np.abs(state_vector)**2), 1.0)

In [25]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


..........
----------------------------------------------------------------------
Ran 10 tests in 0.012s

OK


Info: Input amplitudes (norm^2=2.0000) were not normalized. Normalization has been applied.
Info: Input amplitudes (norm^2=2.0000) were not normalized. Normalization has been applied.
Info: Input amplitudes (norm^2=2.0000) were not normalized. Normalization has been applied.
